In [82]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import TextLoader, DataFrameLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import sys
import os
import pandas as pd
from tqdm import tqdm
from fixthaipdf import clean
from openai import OpenAI


sys.path.append(os.path.abspath('../..'))
import config

## 1. FAQs

In [77]:
faq_df = pd.read_pickle("../../data/innovestx_faq.pkl")
faq_df

,text,context,topic,subtopic
0,ส่งคำสั่งบน TradingView อย่างไร?,สามารถดูวิธีส่งคำสั่งได้ตาม ลิงค์นี https://ww...,faqs,faqs-tradingview
1,ยกเลิกคำสั่งบน TradingView อย่างไร?,1.ผู้ใช้งานนำเมาส์ไปชี้ที่กากบาทของ Order บนกร...,faqs,faqs-tradingview
2,บัญชีที่เชื่อมต่อกับ TradingView คือบัญชีอะไร?,Cash Balance,faqs,faqs-tradingview
3,ทำไมต้องเทรดผ่าน Trading View,1.เพราะคุณสามารถดูราคาหุ้นไทยได้แบบ real time ...,faqs,faqs-tradingview
4,เทรดผ่าน TradingView ค่า Commission เท่าไหร่,ค่าคอมมิชชั่น เท่ากับ 0.157%,faqs,faqs-tradingview
...,...,...,...,...
131,การแสดงข้อความบนหน้าจอบน InnovstX แบ่งตาม Cate...,เมื่อมีข้อความใหม่เข้ามาหรือลูกค้ายังไม่ได้กดอ...,faqs,faqs
132,Intelligent Portfolios มีจุดเด่นอย่างไร เหมาะก...,Intelligent Portfolios เป็นบริการบริหารพอร์ตอั...,faqs,faqs
133,รายละเอียดและจุดเด่นของ Robo Advisor / Guru po...,Robo advisor พอร์ตการลงทุนที่ออกแบบได้ตามระดับ...,faqs,faqs
134,เข้าสู่ Streaming ครั้งแรกได้อย่างไร ?,ตรวจสอบอีเมลหัวข้อ “Innovest X : แจ้งอนุมัติกา...,faqs,faqs


In [78]:
loader = DataFrameLoader(faq_df, page_content_column="text")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
faq_texts = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings(openai_api_key=config.OPENAI_API_KEY)
# faq = FAISS.from_documents(texts, embeddings)
# faq.save_local("frequently_asked_questions")

## 2. Product User Guideline

In [79]:
pug_text_df = pd.read_pickle("../../data/innovestx_pug.pkl")
pug_img_df  = pd.read_pickle("../../data/innovestx_pug_img.pkl")
pug_img_df.columns = ["text",'context','topic','subtopic']

pug_df = pd.concat([pug_text_df, pug_img_df])
pug_df['subtopic'] = pug_df['subtopic'].str.replace('innovestx_pug_','')
pug_df['context']  = pug_df['context'].str.replace('/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/','')
pug_df


,text,context,topic,subtopic
0,Digital Asset : การถอนเงินบาท\n\nการถอนเงินบาท...,data/txt/innovestx_pug_DA-manual-01.txt,pug,DA-manual-01
1,"ขั้นตอนการเปิดบัญชี TFEX, DRx และระบบซื้อขายหล...",data/txt/innovestx_pug_Manual-TFEX.txt,pug,Manual-TFEX
2,เติมเงินเข้าบัญชีบริหารเงินสดผ่าน QR Code\n\nเ...,data/txt/innovestx_pug_topup-qrcode.txt,pug,topup-qrcode
3,วิธีการถอนเหรียญ\n\nวิธีการถอนเหรียญ 1. เข้าเว...,data/txt/innovestx_pug_DA-manual-webtrade08.txt,pug,DA-manual-webtrade08
4,ดูรายละเอียดพอร์ตลงทุน (Real time)\n\nดูรายละเ...,data/txt/innovestx_pug_Real-time.txt,pug,Real-time
...,...,...,...,...
460,"The image depicts an ""Amendment Form"" from ""Fu...",data/img/innovestx_pug_sips-ats-account__21.png,pug,sips-ats-account
461,The image is a screenshot of a cryptocurrency ...,data/img/innovestx_pug_DA-manual-webtrade06__0...,pug,DA-manual-webtrade06
462,The image is a screenshot of an iPhone lock sc...,data/img/innovestx_pug_Notification-Tools__08....,pug,Notification-Tools
463,The image shows a section of a mobile applicat...,data/img/innovestx_pug_Intelligent-Portfolios_...,pug,Intelligent-Portfolios


In [80]:
loader = DataFrameLoader(pug_df, page_content_column="text")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
pug_texts = text_splitter.split_documents(documents)
pug_texts

# embeddings = OpenAIEmbeddings(openai_api_key=config.OPENAI_API_KEY)
# pug = FAISS.from_documents(texts, embeddings)
# pug.save_local("product_user_guidelines")

[Document(metadata={'context': 'data/txt/innovestx_pug_DA-manual-01.txt', 'topic': 'pug', 'subtopic': 'DA-manual-01'}, page_content='Digital Asset : การถอนเงินบาท'),
 Document(metadata={'context': 'data/txt/innovestx_pug_DA-manual-01.txt', 'topic': 'pug', 'subtopic': 'DA-manual-01'}, page_content='การถอนเงินบาทจะมีอยู่ 2 ประเภทดังนี้ 1. ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 2. ถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 01 ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 1. เลือก "สินทรัพย์ดิจิทัล" 2. เลือก "กระเป๋า" 3. เลือก "บัญชีสินทรัพย์ดิจิทัล" 4. เลือก "ถอนเงิน" 5. เลือกบัญชี 6. เลือกบัญชีบริหารเงินสด 7. ระบุจำนวนเงินที่ต้องการถอนไปยังบัญชีบริหารเงินสดโดยมีขั้นต่ำอยู่ที่ 2,000 บาท 8. ตรวจสอบข้อมูลและเลือก "ยืนยัน" 9. ใส่รหัส PIN 10.'),
 Document(metadata={'context': 'data/txt/innovestx_pug_DA-manual-01.txt', 'topic': 'pug', 'subtopic': 'DA-manual-01'}, page_content='2,000 บาท 8. ตรวจสอบข้อมูลและเลือก "ยืนยัน" 9. ใส่รหัส PIN 10. ได้รับราย

## 3. Private Fund Assitance

### 3.1 Landing Page

In [81]:
loader = TextLoader("../../data/innovestx_pfa_landing.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
pfa_la_texts = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings(openai_api_key=config.OPENAI_API_KEY)
# pfa = FAISS.from_documents(texts, embeddings)
# pfa.save_local("private_fund_assistant")

### 3.2 PDFs

In [51]:
pdf_path = "../../data/pdf"
pdfs = [os.path.join(pdf_path, pdf) for pdf in os.listdir(pdf_path) if pdf.endswith(".pdf")]
pdfs

['../../data/pdf/innovestx_pfa_202406-invx-global-alpha-fund-fact-sheet-(th)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-vn-alpha-fund-fact-sheet-(en)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_i-code.pdf',
 '../../data/pdf/innovestx_pfa_report_icode.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-id-alpha-fund-fact-sheet-(en)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-th-alpha-fund-fact-sheet-(en)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-vn-alpha-fund-fact-sheet-(th)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-global-alpha-fund-fact-sheet-(en)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-th-alpha-fund-fact-sheet-(th)-june-2024.pdf',
 '../../data/pdf/innovestx_pfa_i-code-policy.pdf',
 '../../data/pdf/innovestx_pfa_202406-invx-id-alpha-fund-fact-sheet-(th)-june-2024.pdf']

In [52]:
from openai import OpenAI
cleaner = OpenAI(api_key=config.OPENAI_API_KEY)


def clean_thaitext(text, cleaner):
    completion = cleaner.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a language model assistant helping a user clean a Thai text. The output should be only the corrected Thai text without any other information."},
        {"role": "user", "content": "I have a Thai text that I want to clean.\n\nThai text: " + text + "\n\nOutput:"},
    ]
    )
    return completion.choices[0].message.content

In [53]:
all_texts = []
for pdf in pdfs:
    print(pdf)
    loader = PyPDFLoader(pdf)
    documents = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
    texts = text_splitter.split_documents(documents)
    for page in tqdm(texts, desc = 'inner', leave=False):
        text = page.page_content
        cleaned_text = clean_thaitext(text, cleaner)
        page.page_content = cleaned_text
    all_texts += texts
    print(len(all_texts), texts[0].page_content[:100])

../../data/pdf/innovestx_pfa_202406-invx-global-alpha-fund-fact-sheet-(th)-june-2024.pdf


15 กองทุนส่วนบุคคลโมเดล Global Alpha ("Global Alpha")ข้อมูลณ วันที่ 30 มิถุนายน 2567
กลยุทธ์การลงทุนราย
../../data/pdf/innovestx_pfa_202406-invx-vn-alpha-fund-fact-sheet-(en)-june-2024.pdf


27 Private fund model VN Alpha (“VN Alpha”)
 Data as of 30 June 2024
Front-end fee: 1.500%
Management f
../../data/pdf/innovestx_pfa_i-code.pdf


34 บริษัทหลักทรัพย์อินโนเวสท์ เอกซ จำกัด ในทะแยผู้บริหารจัดการลงทุนตระหนักถึงห้ำที่ที่ต้องบริหารจัดการเ
../../data/pdf/innovestx_pfa_report_icode.pdf


47 รายงานผลการติดตามดูแลการจัดการกองทุนส่วนบุคคลประจำปี 2565

สืบเนื้องจาก บริษัทหลักทรัพย์อิโนเวสท์เอก
../../data/pdf/innovestx_pfa_202406-invx-id-alpha-fund-fact-sheet-(en)-june-2024.pdf


...

82 กองทุนส่วนบุคคลโมเดล VN Alpha ("VN Alpha") ข้อมูลณ 30 มิถุนายน 2567
ค่าธรรมเนียมการจัดตั้งกองทุน: 1.
../../data/pdf/innovestx_pfa_202406-invx-global-alpha-fund-fact-sheet-(en)-june-2024.pdf


92 Private fund model Global Alpha (“Global Alpha”) 
ค่าธรรมเนียมด้านหน้า: 1.500% 
ค่าบริหารจัดการ: 1.5
../../data/pdf/innovestx_pfa_202406-invx-th-alpha-fund-fact-sheet-(th)-june-2024.pdf


107 ค่าธรรมเนียมการจัดตั้งกองทุน: 1.000%
ค่าธรรมเนียมการจัดการ: 1.500%
ค่าธรรมเนียมผู้ดูแลผลประโยชน์: 0.
../../data/pdf/innovestx_pfa_i-code-policy.pdf


156 นโยบายธรรมาภิบาลการลงทุน  
เพื่อการจัดการกองทุนส่วนบุคคล  
(Investment Governance Code: I Code)  
ฉบ
../../data/pdf/innovestx_pfa_202406-invx-id-alpha-fund-fact-sheet-(th)-june-2024.pdf


172 กองทุนส่วนบุคคลโมเดล ID Alpha (“ID Alpha”)ข้อมูลณวันที่ 30 มิถุนายน 2567
กลยุทธ์การลงทุนรายละเอียด I


#### Save private_fund_assistance PDF

In [56]:
sources = []
page_numbers = []
contents = []
for page in all_texts:
    sources.append(page.metadata['source'])
    page_numbers.append(page.metadata['page'])
    contents.append(page.page_content)


In [59]:
all_texts

[Document(metadata={'source': '../../data/pdf/innovestx_pfa_202406-invx-global-alpha-fund-fact-sheet-(th)-june-2024.pdf', 'page': 0}, page_content='กองทุนส่วนบุคคลโมเดล Global Alpha ("Global Alpha")ข้อมูลณ วันที่ 30 มิถุนายน 2567\nกลยุทธ์การลงทุนรายละเอียด Global Alpha\nผู้จัดการกองทุน: พสุวุฒิ วิไลนิรันดร์, CFA\nวันจัดตั้งกองทุน: 15 มิถุนายน 2566\nมูลค่าทรัพย์สินสุทิ: 487 ล้านบาท\nสกุลเงินหลัก: บาท (THB)\nสกุลเงินลงทุน: ดอลลาร์สหรัฐฯ (USD) และยูโร (EUR)\nป้องกันความเสี่ยงค่าเงิน: ตามดุลยพินิจ\nดัชนีชี้วัด: MSCI World Index\nหลักทรัพย์ถือครอง: 7-15 บริษัท'),
 Document(metadata={'source': '../../data/pdf/innovestx_pfa_202406-invx-global-alpha-fund-fact-sheet-(th)-june-2024.pdf', 'page': 0}, page_content='ป้องกันความเสี่ยงค่าเงิน: ตามดุลยพินิจ\nดัชนีชี้วัด: MSCI World Index\nหลักทรัพย์ถือครอง: 7-15 บริษัท\nรูปแบบการจัดการ: บัญชีแยกรายบุคคล\nการแจ้งคืนผลประโยชน์: 30 วันทำการล่วงหน้า\nการคำนวณ NAV: รายวัน\nการรายงานผลดำเนินงาน: รายเดือน\nนโยบายการจ่ายเงินปันผล: นากลับ ไปลงทุน\nผู้ดูแลผลประ

In [57]:
df = pd.DataFrame({'source': sources, 'page_number': page_numbers, 'text': contents})
df.to_pickle("../../data/innovestx_pfa_pdf.pkl")      


In [58]:
df

,source,page_number,text
0,../../data/pdf/innovestx_pfa_202406-invx-globa...,0,"กองทุนส่วนบุคคลโมเดล Global Alpha (""Global Alp..."
1,../../data/pdf/innovestx_pfa_202406-invx-globa...,0,ป้องกันความเสี่ยงค่าเงิน: ตามดุลยพินิจ\nดัชนีช...
2,../../data/pdf/innovestx_pfa_202406-invx-globa...,0,ผู้ดูแลผลประโยชน์: ธนาคารดอยช์แบงค์ เอจี\nธนาค...
3,../../data/pdf/innovestx_pfa_202406-invx-globa...,0,โดยผู้จัดการกองทุนบริหารระดับฟองซึ่งเน้นการลงท...
4,../../data/pdf/innovestx_pfa_202406-invx-globa...,0,อ่างอิงกับดัชนีชี้วัดใดแต่ทั้งนี้จะใช้ MSCI Wo...
...,...,...,...
167,../../data/pdf/innovestx_pfa_202406-invx-id-al...,0,ค่าธรรมเนียมที่เก็บจากผู้ลงทุนดังนี้ 1.) ค่าธ...
168,../../data/pdf/innovestx_pfa_202406-invx-id-al...,0,3.) ค่าธรรมเนียมผู้ดูแลผลประโยชน์: ค่าน้ำมันจา...
169,../../data/pdf/innovestx_pfa_202406-invx-id-al...,0,การลงทุนมีความเสี่ยง ผู้ลงทุนควรทำความเข้าใจลั...
170,../../data/pdf/innovestx_pfa_202406-invx-id-al...,0,ค่าธรรมเนียมการจัดการ: 1.500%\nค่าธรรมเนียมผู้...


In [68]:
embeddings = OpenAIEmbeddings(openai_api_key=config.OPENAI_API_KEY)
pfa = FAISS.from_documents(all_texts, embeddings)
pfa.save_local("private_fund_assistant")

### Create Whole database

In [84]:
whole_texts = faq_texts+pug_texts+pfa_la_texts+all_texts
vectorstore = Chroma.from_documents(documents=whole_texts, embedding=embeddings, persist_directory="innovestx_db")